# Day 4 - Part 1: Backtrader 소개와 기본 사용법

## 학습 목표
- 백테스팅이 무엇인지 이해하기
- Backtrader 설치와 기본 구조 파악하기
- Buy & Hold 전략으로 첫 백테스트 실행하기

## 백테스팅이란?
"만약 2년 전부터 이 전략을 썼다면 얼마나 벌었을까?" 를 시뮬레이션하는 것입니다.

**주의**: 과거에 잘 됐다고 미래에도 잘 되는 건 아닙니다!

- 주식, 선물, FX, 암호화폐, 금/원유 등
  - OHLCV 데이터만 있으면 모두 가능
  - 헤지 전략(페어 트레이딩/스프레드 헤지, 지수/ETF 기반 헤지, 선물/옵션 기반 헤지) 가능

- backtrader 는 멀티자산 포트폴리오 & 헤지 전략까지 구현 가능한 범용 백테스트 엔진

## 1. 라이브러리 설치 및 임포트

In [1]:
# 필요한 라이브러리 설치 (주석 해제해서 실행)
# !pip install backtrader yfinance pandas matplotlib

In [2]:
import backtrader as bt
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

print("라이브러리 임포트 완료!")

라이브러리 임포트 완료!


## 2. 데이터 준비하기

삼성전자 주가 데이터를 다운로드합니다.

In [3]:
# 삼성전자 데이터 다운로드 (2020-01-01 ~ 2023-12-31)
ticker = '005930.KS'  # 삼성전자
data = yf.download(ticker, start='2020-01-01', end='2024-12-31', multi_level_index=False)

# 데이터 확인
print(f"\n데이터 shape: {data.shape}")
print(f"기간: {data.index[0]} ~ {data.index[-1]}")
print("\n처음 5행:")
print(data.head())

C:\Users\panis\AppData\Local\Temp\ipykernel_3008\1231455855.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2020-01-01', end='2024-12-31', multi_level_index=False)
[*********************100%***********************]  1 of 1 completed


데이터 shape: (1229, 5)
기간: 2020-01-02 00:00:00 ~ 2024-12-30 00:00:00

처음 5행:
                   Close          High           Low          Open    Volume
Date                                                                        
2020-01-02  47701.652344  48392.980639  47528.820270  47960.900454  12993228
2020-01-03  47960.906250  48911.482770  47442.409966  48392.986486  15422255
2020-01-06  47960.906250  48047.322297  47183.161824  47442.409966  10278951
2020-01-07  48220.167969  48738.664399  48047.335825  48133.751897  10009778
2020-01-08  49084.320312  49602.816654  48306.575801  48565.823971  23501171


In [4]:
# Backtrader용으로 데이터 저장
data.to_csv('samsung_data.csv')
print("samsung_data.csv 파일로 저장 완료!")

samsung_data.csv 파일로 저장 완료!


## 3. Backtrader의 기본 구조

Backtrader는 3가지 요소로 구성됩니다:

1. **Strategy (전략)**: 언제 사고 팔지 규칙을 정의
2. **Cerebro (엔진)**: 백테스트를 실행하는 엔진
3. **Data Feed (데이터)**: 주가 데이터를 넣어줌

## 4. 첫 번째 전략: Buy & Hold

가장 간단한 전략입니다. 처음에 사서 끝까지 보유합니다.

In [5]:
class BuyAndHold(bt.Strategy):
    """
    Buy & Hold 전략
    - 첫날 전체 금액으로 매수
    - 끝까지 보유
    """
    
    def __init__(self):
        self.order = None  # 주문 관리용
        self.start_date = None
        
    def next(self):
        """
        매일 실행되는 함수
        """
        # 첫 바에서만 시작일 저장
        if self.start_date is None:
            self.start_date = self.data.datetime.date(0)
            print(f"시작일자 : {self.start_date}")

        # 아직 주식을 사지 않았다면
        if not self.position:
            # 전체 금액으로 매수
            self.order = self.buy()
            print(f"{self.data.datetime.date(-1)}: 매수 주문!")
    
    def notify_order(self, order):
        """
        주문이 체결되면 실행되는 함수
        """
        if order.status in [order.Completed]:
            if order.isbuy():
                print(f"{self.data.datetime.date(0)}: 매수 체결: {order.executed.price:,.0f}원")

    def stop(self):
        print(f"종료일자 : {self.data.datetime.date(0)}")

## 5. Cerebro 엔진 설정하기

In [6]:
# Cerebro 엔진 생성
cerebro = bt.Cerebro()

# 초기 자금 설정 (1억원)
cerebro.broker.setcash(100_000_000)

# 수수료 설정 (0.1%)
cerebro.broker.setcommission(commission=0.001)

print("Cerebro 엔진 생성 완료!")
print(f"초기 자금: {cerebro.broker.getvalue():,.0f}원")

Cerebro 엔진 생성 완료!
초기 자금: 100,000,000원


## 6. 데이터 추가하기

In [7]:
# CSV 파일에서 데이터 읽기
data_bt = bt.feeds.GenericCSVData(
    dataname='samsung_data.csv',
    dtformat='%Y-%m-%d',
    datetime=0,  # Date 컬럼 위치
    open=1,      # Open 컬럼 위치
    high=2,      # High 컬럼 위치
    low=3,       # Low 컬럼 위치
    close=4,     # Close 컬럼 위치
    volume=5,    # Volume 컬럼 위치
    openinterest=-1  # 없음
)

# Cerebro에 데이터 추가
cerebro.adddata(data_bt)
print("데이터 추가 완료!")

데이터 추가 완료!


## 7. 전략 추가하기

In [8]:
# Buy & Hold 전략 추가
cerebro.addstrategy(BuyAndHold)
print("전략 추가 완료!")

전략 추가 완료!


## 8. 백테스트 실행!

In [9]:
# 시작 전 자금
start_value = cerebro.broker.getvalue()
print(f"\n{'='*50}")
print(f"백테스트 시작!")
print(f"시작 자금: {start_value:,.0f}원")
print(f"{'='*50}\n")

# 백테스트 실행
cerebro.run()

# 종료 후 자금
end_value = cerebro.broker.getvalue()
profit = end_value - start_value
return_pct = (profit / start_value) * 100

print(f"\n{'='*50}")
print(f"백테스트 종료!")
print(f"{'='*50}")
print(f"최종 자금: {end_value:,.0f}원")
print(f"수익금: {profit:,.0f}원")
print(f"수익률: {return_pct:.2f}%")
print(f"{'='*50}\n")


백테스트 시작!
시작 자금: 100,000,000원



시작일자 : 2020-01-02
2024-12-30: 매수 주문!
2020-01-03: 매수 체결: 47,961원
종료일자 : 2024-12-30

백테스트 종료!
최종 자금: 100,004,419원
수익금: 4,419원
수익률: 0.00%



## 9. 결과 시각화

In [38]:
print("interactive:", plt.isinteractive())
print("backend:", plt.get_backend())

interactive: True
backend: module://matplotlib_inline.backend_inline


In [36]:
# 기본 plot
# iplot = False : cerebro 와 vscode jupyter backend 충돌로 외부 이미지 창 띄우기
# 외부 이미지 창을 닫아야 다음 셀로 실행이 넘어감
%matplotlib tk
# 포트폴리오 가치 그래프
cerebro.plot(style='candlestick', barup='red', bardown='blue', iplot=False)

[[<Figure size 640x480 with 4 Axes>]]

In [37]:
# 실행 후 backend 바꾸기
%matplotlib inline

## 정리

배운 내용:
- 백테스팅은 과거 데이터로 전략을 시뮬레이션하는 것
- Backtrader는 Strategy + Cerebro + Data로 구성
- Buy & Hold는 가장 간단한 전략

주의사항:
- 과거 결과 ≠ 미래 결과
- 수수료와 슬리피지를 꼭 고려해야 함